In [1]:
# ffmpeg
!apt -y install ffmpeg
# 파이썬 패키지
!pip -q install faster-whisper python-dotenv openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [5]:
%%writefile stt.py
import os, tempfile, subprocess
from transformers import pipeline
import torch

# 1) ffmpeg로 비디오→WAV(PCM 16bit)만 뽑기
def extract_audio_pcm16(input_video: str) -> str:
    td = tempfile.mkdtemp(prefix="stt_")
    wav_path = os.path.join(td, "audio.wav")
    cmd = ["ffmpeg", "-y", "-i", input_video, "-vn", "-c:a", "pcm_s16le", wav_path]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return wav_path  # 임시 wav 경로 반환

# 2) Whisper 파이프라인(긴 파일도 청크로 자동 처리)
ASR_MODEL  = "openai/whisper-small"  # 필요시 medium/large
ASR_DEVICE = 0 if torch.cuda.is_available() else -1
asr = pipeline(
    "automatic-speech-recognition",
    model=ASR_MODEL,
    device=ASR_DEVICE,
    chunk_length_s=30,
    generate_kwargs={"task":"transcribe", "language":"<|ko|>"}
)

# 3) 엔드투엔드: 영상 경로만 넣으면 텍스트 반환
def transcribe_video(video_path: str) -> str:
    wav_path = extract_audio_pcm16(video_path)
    try:
        text = asr(wav_path)["text"]  # 전사 결과만 뽑아서
        return text                   # 그대로 반환
    finally:
        # 임시 wav 정리(영상 원본은 그대로 둠)
        try:
            os.remove(wav_path)
            os.rmdir(os.path.dirname(wav_path))
        except Exception:
            pass



Writing stt.py


In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/온라인해커톤

/content/drive/MyDrive/Colab Notebooks/온라인해커톤


In [7]:
import os
with open('./key/openai_api_key', 'r') as f:
    api_key = f.read().strip()

os.environ['OPENAI_API_KEY'] = api_key

In [25]:
from typing import Dict, List, Optional
from pydantic import BaseModel, Field
from typing import Literal
# UserProfile 클래스 정의
class UserProfile(BaseModel):
    mobility_issue: bool = Field(default=True, description="거동 불편 여부")
    living_arrangement: Literal['alone', 'with_family'] = Field(default='alone', description="가족 동거 여부")
    wake_up_time: str = Field(default="07:00", description="기상 시간")
    bed_time: str = Field(default="21:00", description="취침 시간")


In [29]:
# next_pipeline.py
import os
from typing import Dict, List
from openai import OpenAI
from stt import transcribe_video
from typing import Optional
import json


# 1) 키 로드
with open('./key/openai_api_key', 'r') as f:
    api_key = f.read().strip()

# 2) 환경 변수 등록
os.environ['OPENAI_API_KEY'] = api_key

# 3) 클라이언트 생성
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


# === 1) 텍스트 감정 ===
EMO_LABELS = ["happy","neutral","sad","angry","fear","disgust","surprise"]

def classify_text_emotion(text: str) -> Dict:
    prompt = f"""
다음 한국어 텍스트의 감정을 분석하세요.
- 주 감정(primary): 가장 강하게 나타나는 감정 1개
- 보조 감정(secondary): 함께 감지되는 부가적인 감정 0~2개
  * 신뢰도 0.3 이상인 경우만 포함
  * 주 감정과 충분히 구별되는 경우만 포함
  * 없으면 빈 리스트

가능한 감정: {", ".join(EMO_LABELS)}

JSON으로만 답하세요:
{{
  "primary": {{"label": "감정명", "confidence": 0.0~1.0}},
  "secondary": [
    {{"label": "감정명", "confidence": 0.0~1.0}},
    ...
  ]
}}

텍스트: {text}
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a precise emotion classifier that detects multiple emotional layers."},
            {"role": "user", "content": prompt},
        ],
    )
    content = resp.choices[0].message.content
    data = json.loads(content)

    # 응답 검증 및 기본값 설정
    if "primary" not in data:
        data["primary"] = {"label": "neutral", "confidence": 0.5}
    if "secondary" not in data:
        data["secondary"] = []

    # 보조 감정 필터링 (신뢰도 낮은 것 제거)
    data["secondary"] = [s for s in data["secondary"] if s.get("confidence", 0) >= 0.3][:2]

    return data

# === 2) 얼굴 감정 ===
def run_face_emotion(video_path: str) -> List[Dict]:
    """
    TODO: 실제 얼굴/표정 모델 연결.
    지금은 빈 리스트 반환 (텍스트만 사용).
    """
    return []  # 예: [{"label":"happy","confidence":0.72}]

# === 3) 융합 로직 (얼굴 우선, 근소 차이는 텍스트 허용) ===
def fuse_emotion(video_preds: List[Dict], text_pred: Dict,
                 min_conf=0.35, tiny_gap=0.05) -> Dict:
    """
    영상과 텍스트 감정을 융합
    - 주 감정은 기존 로직 유지
    - 보조 감정은 텍스트에서만 가져옴
    """
    # 기존 로직으로 주 감정 결정
    v_top = max(video_preds, key=lambda x: x["confidence"]) if video_preds else None
    t_primary = text_pred.get("primary", {"label": "neutral", "confidence": 0.5})

    # 주 감정 선택
    if v_top and v_top["confidence"] >= min_conf:
        if abs(v_top["confidence"] - t_primary["confidence"]) <= tiny_gap:
            primary = {"label": t_primary["label"],
                      "confidence": round(t_primary["confidence"], 2),
                      "source": "text_narrow_gap"}
        else:
            primary = {"label": v_top["label"],
                      "confidence": round(v_top["confidence"], 2),
                      "source": "video"}
    else:
        primary = {"label": t_primary["label"],
                  "confidence": round(t_primary["confidence"], 2),
                  "source": "text"}

    # 보조 감정은 텍스트에서 가져오되, 주 감정과 중복 제거
    secondary = []
    for s in text_pred.get("secondary", []):
        if s["label"] != primary["label"]:
            secondary.append({
                "label": s["label"],
                "confidence": round(s["confidence"], 2)
            })

    return {
        "primary": primary,
        "secondary": secondary
    }

# === 4) 공감 멘트 ===
def build_empathy(fused: Dict, text: str) -> str:
    # 감정 설명 문자열 생성
    emotion_desc = f"주 감정: {fused['primary']['label']} (신뢰도 {fused['primary']['confidence']})"

    if fused.get('secondary'):
        secondary_labels = [f"{s['label']}({s['confidence']})" for s in fused['secondary']]
        emotion_desc += f"\n보조 감정: {', '.join(secondary_labels)}"

    prompt = f"""
아래 정보를 바탕으로 고령자에게 공감과 격려를 전하는 짧은 문단을 만드세요.
- 존댓말, 쉬운 어휘, 200자 이내
- 이모지/특수기호/괄호 금지
- 구조: 감사 → 감정요약 → 공감/격려 → 맞춤 제안
- 주 감정을 중심으로 하되, 보조 감정도 자연스럽게 언급

{emotion_desc}
말씀 요지: {text[:120]}
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[
            {"role":"system","content":"You are a warm, emotionally perceptive Korean caregiver assistant."},
            {"role":"user","content":prompt}
        ],
    )
    return resp.choices[0].message.content.strip()


# === 5) 감정 기반 미션 ===
def make_emo_mission(fused: Dict, text: str, user_profile: Optional[UserProfile] = None) -> Dict:
    """감정 기반 미션 생성 (선택적으로 사용자 프로필 고려)"""

    # 감정 설명
    emotion_desc = f"주 감정: {fused['primary']['label']} (신뢰도 {fused['primary']['confidence']})"
    if fused.get('secondary'):
        secondary_labels = [s['label'] for s in fused['secondary']]
        emotion_desc += f"\n보조 감정: {', '.join(secondary_labels)}"

    # 사용자 프로필이 있으면 추가
    user_context = ""
    if user_profile:
        user_context = f"""

사용자 상황:
- 거동 불편 여부: {'예 (이동 제한, 앉아서 가능한 활동 위주)' if user_profile.mobility_issue else '아니오 (자유로운 이동 가능)'}
- 거주 형태: {'독거 (혼자서 안전하게 할 수 있는 활동)' if user_profile.living_arrangement == 'alone' else '가족과 함께 (가족 참여 가능)'}

위 상황을 반드시 고려하여 적합한 미션을 제안하세요.
"""

    prompt = f"""
고령자 친화 '감정 기반 미션' 1개를 만들어주세요.
- 존댓말, 쉬운 어휘, 이모지/특수기호/괄호 금지
- 의료·법률·위험 활동·개인정보 요구 금지
- 출력은 JSON만: title(12자 이내), steps(60자 이내), duration(예: 3분), difficulty(very_easy|easy)
{user_context}
{emotion_desc}
말씀 요지: {text[:120]}
"""

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        temperature=0.3,
        messages=[
            {"role": "system", "content": "You write short, safe, elder-friendly tasks in Korean."},
            {"role": "user", "content": prompt},
        ],
    )
    content = resp.choices[0].message.content
    data = json.loads(content)
    return data

# === 6) 전체 실행 ===
# === 기존의 run_full_pipeline을 두 개로 분리 ===

def run_core_pipeline(video_path: str) -> Dict:
    """
    핵심 파이프라인 (미션 제외)
    STT → 감정 분석 → 융합 → 공감 멘트
    """
    # 1) STT
    text = transcribe_video(video_path)

    # 2) 감정들
    txt_pred = classify_text_emotion(text)  # {"primary": {...}, "secondary": [...]}
    vid_preds = run_face_emotion(video_path)  # [] (추후 구현)

    # 3) 융합
    fused = fuse_emotion(vid_preds, txt_pred)

    # 4) 공감 멘트만 생성
    empathy = build_empathy(fused, text)

    return {
        "stt_text": text,
        "text_emotion": txt_pred,
        "video_emotions": vid_preds,
        "fused_emotion": fused,
        "empathy": empathy
        # "mission" 없음!
    }


def generate_emo_mission(core_result: Dict, user_profile: Optional[UserProfile] = None) -> Dict:
    """
    미션 생성 (사용자 프로필 선택적 고려)
    """
    mission = make_emo_mission(
        core_result["fused_emotion"],
        core_result["stt_text"],
        user_profile  # 그대로 전달
    )
    return mission

